In [15]:
import numpy as np

images = np.load('trnImage.npy')
labels = np.load('trnLabel_coarse.npy')
#labels_fine = np.load('trnLabel_fine.npy')

test_images = np.load('tstImage.npy')
test_labels = np.load('tstLabel_coarse.npy')
#test_labels_fine = np.load('tstLabel_fine.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Coarse Shape: {labels.shape}')

Images Shape: (32, 32, 3, 50000)
Train Labels Coarse Shape: (50000,)


In [16]:
import tensorflow as tf

In [17]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))
labels_reshaped = labels.reshape(50000, 1)

#labels_fine_reshaped = labels_fine.reshape(50000, 1)


x_train = train_images_reshaped
y_train = labels_reshaped
#y_train = labels_fine_reshaped

x_test = test_images_reshaped
y_test = test_labels

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3)


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 20 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 20
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

#Defining the parameters
batch_size= 100
epochs=100
learn_rate=.001

((35000, 32, 32, 3), (35000, 20))
((15000, 32, 32, 3), (15000, 20))
((10000, 32, 32, 3), (10000, 20))


In [21]:
tf.keras.applications.resnet.preprocess_input(x_train)
tf.keras.applications.resnet.preprocess_input(y_train)
tf.keras.applications.resnet.preprocess_input(x_val)
tf.keras.applications.resnet.preprocess_input(y_val)

array([[-103.939, -116.779, -123.68 , ...,    0.   ,    0.   ,    0.   ],
       [-103.939, -116.779, -123.68 , ...,    0.   ,    0.   ,    0.   ],
       [-103.939, -116.779, -123.68 , ...,    0.   ,    0.   ,    1.   ],
       ...,
       [-103.939, -116.779, -123.68 , ...,    0.   ,    1.   ,    0.   ],
       [-103.939, -116.779, -123.68 , ...,    0.   ,    1.   ,    0.   ],
       [-103.939, -116.779, -123.68 , ...,    0.   ,    0.   ,    0.   ]],
      dtype=float32)

In [22]:
resnet_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_tensor=None,
    input_shape=(32,32,3),
    pooling=max,
    classes=20)



In [23]:
resnet_model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [24]:
resnet_model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_val)//batch_size, verbose=1)

Epoch 1/100


ValueError: in user code:

    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lizks\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replic

In [25]:
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar100
import tensorflow as tf
import datetime

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
x_train, x_test = x_train / 255, x_test / 255 # normalize data

numtrain = 500
numvalidate = 100

x_train = x_train[:numtrain, :]
y_train = y_train[:numtrain, :]

x_test = x_test[:numvalidate, :]
y_test = y_test[:numvalidate, :]

model = ResNet50(input_shape=x_train.shape[1:], weights=None)
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'])

log_dir = 'logs\\resnet\\' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    x = x_train,
    y = y_train,
    epochs = 2,
    validation_data = (x_test, y_test),
    callbacks = [tensorboard_callback]
)

169009152/169001437 [==============================] - 78s 0us/step
Epoch 1/2
16/16 [==============================] - 38s 2s/step - loss: 8.4959 - accuracy: 0.0011 - val_loss: 6.2184 - val_accuracy: 0.0100
Epoch 2/2
16/16 [==============================] - 26s 2s/step - loss: 5.2936 - accuracy: 0.0677 - val_loss: 5.5689 - val_accuracy: 0.0100
